In [33]:
import requests
import pandas as pd

# Set the Google Distance Matrix API endpoint
distance_endpoint = 'https://maps.googleapis.com/maps/api/distancematrix/json'

# Set the Google Geocode API endpoint
geocode_endpoint = 'https://maps.googleapis.com/maps/api/geocode/json'

# Set the API key (replace with your own API key)
api_key = 'Your_API'

# Read the list of destinations from the file
with open('dests.txt', 'r') as f:
    destinations = f.read().splitlines()

# Define a function to retrieve the distance, time, latitude, and longitude for a given destination
def get_destination_data(destination):
    # Set the origin and destination cities
    origin = 'Tel Aviv'

    # Set the API parameters for the Distance Matrix API request
    distance_params = {
        'origins': origin,
        'destinations': destination,
        'key': api_key,
        'units': 'metric'
    }

    # Send the Distance Matrix API request and handle exceptions
    try:
        distance_response = requests.get(distance_endpoint, params=distance_params)
        distance_data = distance_response.json()
    except:
        print(f"Error getting distance data for {destination}")
        return None

    # Check if the response contains the expected data and handle exceptions
    try:
        distance = distance_data['rows'][0]['elements'][0]['distance']['value'] / 1000 # Convert meters to kilometers
        duration_minutes = distance_data['rows'][0]['elements'][0]['duration']['value'] // 60 # Convert seconds to minutes
        duration_hours = duration_minutes // 60
        duration_minutes %= 60
        duration = f"{duration_hours} hours and {duration_minutes} minutes"
    except:
        print(f"No distance data found for {destination}")
        return None

    # Set the API parameters for the Geocode API request
    geocode_params = {
        'address': destination,
        'key': api_key
    }

    # Send the Geocode API request and handle exceptions
    try:
        geocode_response = requests.get(geocode_endpoint, params=geocode_params)
        geocode_data = geocode_response.json()
    except:
        print(f"Error getting geocode data for {destination}")
        return None

    # Check if the response contains the expected data and handle exceptions
    try:
        latitude = geocode_data['results'][0]['geometry']['location']['lat']
        longitude = geocode_data['results'][0]['geometry']['location']['lng']
    except:
        print(f"No geocode data found for {destination}")
        return None

    # Return the data as a dictionary
    return {
        'Destination': destination,
        'Distance (km)': distance,
        'Duration': duration,
        'Latitude': latitude,
        'Longitude': longitude
    }

# Use list comprehension to retrieve the data for each destination
data = [get_destination_data(dest) for dest in destinations]

# Remove any None values from the list
data = [d for d in data if d is not None]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

In [34]:
df

,Destination,Distance (km),Duration,Latitude,Longitude
0,Istanbul,1815.227,21 hours and 2 minutes,41.008238,28.978359
1,Amsterdam,4532.541,48 hours and 3 minutes,52.367573,4.904139
2,Valletta,3792.883,50 hours and 50 minutes,35.899237,14.514100
3,Basel,4092.805,44 hours and 2 minutes,47.559599,7.588576
4,Doha,2164.477,22 hours and 38 minutes,25.285447,51.531040


In [35]:
# Sort the DataFrame by "Distance (km)" column in descending order
df_sorted = df.sort_values(by="Distance (km)", ascending=False)

# Get the top 3 rows and select only the "Destination" and "Distance (km)" columns
top_3_cities = df_sorted.head(3)[["Destination", "Distance (km)"]]

# Print the result
print(top_3_cities)

  Destination  Distance (km)
1   Amsterdam       4532.541
3       Basel       4092.805
2    Valletta       3792.883
